In [1]:
import json
from psycopg2 import connect
import pandas as pd
from datetime import datetime

from utils.extraction import get_currencies_from
from utils.validation import (
    validate_shape,
    validate_columns,
    validate_no_duplicates,
    validate_no_null,
    validate_date_column,
    validate_currency_column,
    validate_rate_column
)

from utils.load_to_db import load_df_to_db

# Download data
currencies = get_currencies_from('MXN', '2021-01-01')
currencies_df = pd.DataFrame(currencies)

# Get monthly mean exchange rate by currency
currencies_df['date'] = currencies_df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
currencies_df = currencies_df.set_index('date').groupby('currency').resample('MS').mean().reset_index() 
currencies_df['date'] = currencies_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

# Rename columns
currencies_df.columns = ['currency', 'rate_date', 'rate']

# Data validation
validate_shape(currencies_df)
validate_columns(currencies_df, ['currency', 'rate_date', 'rate'])
validate_no_duplicates(currencies_df, ['currency', 'rate_date'])
validate_no_null(currencies_df)
validate_date_column(currencies_df, 'rate_date')
validate_currency_column(currencies_df, 'currency')
validate_rate_column(currencies_df, 'rate')

## Load data to DB
# Load credentials
with open('creds.json', 'r') as f:
    creds = json.load(f)

# Create conection
conn = connect(**creds)

# Load data to DB
load_df_to_db(currencies_df, 'javier_santibanez_coderhouse.exchange_rates_mxn', conn)

# ## Validate data in DB
# # Crear un cursor
# cursor = conn.cursor()

# try:
#     cursor.execute("SELECT * FROM javier_santibanez_coderhouse.exchange_rates_mxn")
#     colnames = [desc[0] for desc in cursor.description]
#     res = pd.DataFrame(cursor.fetchall(), columns=colnames)
# except psycopg2.Error as e:
#     print("Error al ejecutar el query:", e)
# finally:
#     cursor.close()

# display(res.head(3))
# display(res.tail(3))
# print(res.shape)

Data insertion completed.
